<a href="https://colab.research.google.com/github/hongdoi/datacracy-atom--doi-thi-hong-/blob/main/Home_Assignment_3_Google_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

In [16]:
!ls

env_variable_slack.json  home-assignment-3-ea805a06baea.json  sample_data


In [19]:
with open('env_variable_slack.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [20]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [21]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

In [25]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [26]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [27]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'home-assignment-3-ea805a06baea.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [28]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1PZJQTaSHoXn3FhQ7kvpU-NFKmIaP_jxh262dgggnK9M' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

In [29]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [44]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': True,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 65,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

In [59]:
for i in range(len(channel_ls)): 
    if(channel_ls[i]['name']=='atom-assignment2'): print(channel_ls[i]['id'])

C021FSDN7LJ


In [60]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [61]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [63]:
msg_ls[22]

{'attachments': [{'fallback': 'Google Colaboratory',
   'from_url': 'https://colab.research.google.com/drive/1zWbE-TC1Yw5TBUen4ezrP51sV68ezhfD?usp=sharing',
   'id': 1,
   'image_bytes': 4534,
   'image_height': 250,
   'image_url': 'https://colab.research.google.com/img/colab_favicon_256px.png',
   'image_width': 250,
   'original_url': 'https://colab.research.google.com/drive/1zWbE-TC1Yw5TBUen4ezrP51sV68ezhfD?usp=sharing',
   'service_icon': 'https://colab.research.google.com/img/favicon.ico?vrz=colab-20210429-060047-RC00_371090719',
   'service_name': 'colab.research.google.com',
   'title': 'Google Colaboratory',
   'title_link': 'https://colab.research.google.com/drive/1zWbE-TC1Yw5TBUen4ezrP51sV68ezhfD?usp=sharing'}],
 'blocks': [{'block_id': '7wmK',
   'elements': [{'elements': [{'text': 'Em gửi Assignment 1 ạ:\n',
       'type': 'text'}],
     'type': 'rich_text_section'},
    {'elements': [{'elements': [{'text': 'Task 1: ', 'type': 'text'},
        {'text': 'https://docs.google

In [64]:
not_learners_id = ['U01BE2PR6LU']

In [66]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 04:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 08:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 06:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/auslynnguyen/


In [75]:
git_dict = {'time_stampt':[], 'user':[], 'reply_users_count':[],'reply_users':[],'latest_reply':[],'github':[]}
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      git_dict['time_stampt'].append(ts)
      git_dict['user'].append(user)
      git_dict['reply_users_count'].append(reply_users_count)
      git_dict['reply_users'].append(reply_users)
      git_dict['latest_reply'].append(latest_reply)
      git_dict['github'].append(github)


In [76]:
git_dict = pd.DataFrame(git_dict) ## Dùng pandas để convert dictionaries thành bảng
sheet_index_no = 1
spreadsheet_key = '1PZJQTaSHoXn3FhQ7kvpU-NFKmIaP_jxh262dgggnK9M' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(1) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, git_dict)